<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [1]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [11]:
#Ignore warnings and set max row display option
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 300)
pd.set_option('display.min_rows',300)

In [3]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [4]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [5]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [6]:
#Filter securities with at least median observations in the period. This is not a great way to eliminate observations, so I would be cautious in doing this. There are better ways to filter out insufficient data
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >= int(pd.to_numeric(observation_df['COUNT(PERMNO)']).median())]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [7]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#These dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month so that we can append on index
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()

#Convert macro factors date column to type datetime64 and drop the non-datetime64 column
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)

#Inner-join macro factors dataframe with universe dataframe, using 'date' column as index
universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#Ensure no errors in the returns column (there have been some instances where returns have taken on non-numeric values)
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

#Here, we select our required columns in proper order (see function documentation for more info) and get rid of extra columns that we will not use
universe_df_with_external_factors_filtered = universe_df_with_external_factors[["PERMNO","date","RET", "Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]]

#Remove duplicate rows from universe (monthly returns) dataframe to avoid faulty data 
universe_df_with_external_factors_filtered = universe_df_with_external_factors_filtered.drop_duplicates()

display(universe_df_with_external_factors_filtered)

,PERMNO,date,RET,"Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted",CPI (USACPALTT01CTGYM),UNRATE
0,10026,2017-12-01,0.007743,3.762747,2.091918,4.1
1,10028,2017-12-01,-0.104808,3.762747,2.091918,4.1
2,10032,2017-12-01,-0.028635,3.762747,2.091918,4.1
3,10044,2017-12-01,0.024296,3.762747,2.091918,4.1
4,10051,2017-12-01,NaN,3.762747,2.091918,4.1
...,...,...,...,...,...,...
234583,93426,2021-01-01,0.016201,8.270387,1.365916,6.4
234584,93427,2021-01-01,0.017399,8.270387,1.365916,6.4
234585,93429,2021-01-01,-0.014927,8.270387,1.365916,6.4
234586,93434,2021-01-01,0.194539,8.270387,1.365916,6.4


In [13]:
#Defining the factor model that takes a dataframe of the required columns (unique identifier, date/index column, returns columns, and a set of factors)
def olsfactormodel(df_attached, lookbackwindow:int, returnsvec:bool):

    #Assign passed dataframe to new dataframe
    df = df_attached.copy(deep=True)
    
    #We generate our list of unique tickers using the column in the unique identifier position (0, or leftmost column)
    uniqueTickerList = df.iloc[:,0].unique()
    
    #Initialize large df to drop results of regression for each security at each regression date
    containerdf = pd.DataFrame()
                
    #Set our lookback window to 24 periods (months in this case). This means that we will run a regression for each period (after the first 24 months) using the previous 24 months as data
    LookBack_Window=lookbackwindow
    
    #We will now create a new set of columns for the Beta and P-value for each of our factors. We will do this by iterating through each factor and columns for the beta coefficients
    factorlist = list(df.columns[3:])
    
    #For each factor, define an empty column to hold the corresponding coefficient
    for factor in factorlist: 
        df["Beta_" + factor] = 0


    #For each factor, define an empty column to hold the corresponding t-stat/p-value
    for factor in factorlist: 
        df["P-Value_" + factor] = 0
    
    #Initialize global parameter(s) regardless of factor count
    df["R_squared"] = 0
    df["Constant B0"] = 0

    #For each identifier (ticker), generate a dataframe from the broader dataframe that 
    for ticker in uniqueTickerList:
        
        #For each unique identifier (ticker), we create a dataframe with observations from that particular identifier
        ticker_specific_universe_df = df.loc[df.iloc[:,0] == ticker]
        
        #We then sort our date/index column in position 1 to ensure our date is ascending from the earliest available
        ticker_specific_universe_df = ticker_specific_universe_df.sort_values(by= ticker_specific_universe_df.columns[1])

        #Shift our returns data back by one period. This way, we regress "t" factors to "t+1" returns, and our betas become forecasts
        ticker_specific_universe_df['RET'] = ticker_specific_universe_df['RET'].shift(-1)

        if returnsvec == True:
            ticker_specific_universe_df = ticker_specific_universe_df.tail(LookBack_Window+1)

        #For each lookback window span, train an OLS and collect the results
        for x in range(0, (len(ticker_specific_universe_df)-LookBack_Window)):
            
            # Define the independent variables (X) and dependent variable (Y). X's are defined by our factor columns and our returns are defined in our third left-most column (position 2)
            X = ticker_specific_universe_df[factorlist][x:x+LookBack_Window]
            Y = ticker_specific_universe_df[ticker_specific_universe_df.columns[2]][x:x+LookBack_Window]
    
            #Add a constant term to the independent variables, check impact
            X = sm.add_constant(X)
        
            #Fit the linear regression model
            model = sm.OLS(Y, X)
            results = model.fit()
            
            #Place our regression coefficients into their appropriate columns
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], "Constant B0"] = results.params[0]
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], "R_squared"] = results.rsquared
    
            #We do the same dynamically using our list of factors and placing into the appropriate factor column
            #Define an index (starting position for inserting regression params)
            i = 3 + len(factorlist) - 1
            
            for index in range(len(factorlist)):
                #increase our indices by one for each factor
                index+=1
                i+=1
                j = i+len(factorlist)
                
                ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], ticker_specific_universe_df.columns[i]] = results.params[index]
                ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+LookBack_Window], ticker_specific_universe_df.columns[j]] = results.pvalues[index]

        #Add ticker dataframe to larger container dataframe
        if returnsvec==True:
            containerdf = pd.concat([containerdf, ticker_specific_universe_df[ticker_specific_universe_df["date"]==np.max(ticker_specific_universe_df.date)]], ignore_index=True)
        else:
            containerdf = pd.concat([containerdf, ticker_specific_universe_df], ignore_index=True)
    return containerdf

In [15]:
returnsdf = olsfactormodel(universe_df_with_external_factors_filtered,36,False)
display(returnsdf)

,PERMNO,date,RET,"Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted",CPI (USACPALTT01CTGYM),UNRATE,"Beta_Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted",Beta_CPI (USACPALTT01CTGYM),Beta_UNRATE,"P-Value_Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted",P-Value_CPI (USACPALTT01CTGYM),P-Value_UNRATE,R_squared,Constant B0
0,10026,2017-12-01,-0.088191,3.762747,2.091918,4.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,10026,2018-01-01,-0.029688,2.590361,2.042188,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,10026,2018-02-01,0.019951,3.685589,2.175117,4.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,10026,2018-03-01,0.006224,1.852684,2.321362,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10026,2018-04-01,0.030638,1.741385,2.406737,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,10026,2018-05-01,0.079791,1.782396,2.725657,3.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,10026,2018-06-01,-0.049256,2.879417,2.800613,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,10026,2018-07-01,0.003725,2.878295,2.889804,3.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,10026,2018-08-01,0.040137,2.698940,2.651383,3.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,10026,2018-09-01,0.034926,0.813265,2.240983,3.7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
